In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/TIU-con/LED

/content/drive/MyDrive/TIU-con/LED


In [3]:
#Checking predicted results csv files total rows.
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/TIU-con/LED/LED_result.csv")
#data

In [4]:
data_final=data[['article', 'abstract', 'predicted_abstract']]
data_final['abstract'][0]

" research on the implications of anxiety in parkinson 's disease ( pd ) has been neglected despite its prevalence in nearly 50% of patients and its negative impact on quality of life . \n previous reports have noted that neuropsychiatric symptoms impair cognitive performance in pd patients ; however , to date , no study has directly compared pd patients with and without anxiety to examine the impact of anxiety on cognitive impairments in pd . \n this study compared cognitive performance across 50 pd participants with and without anxiety ( 17 pda+ ; 33 pda ) , who underwent neurological and neuropsychological assessment . \n group performance was compared across the following cognitive domains : simple attention / visuomotor processing speed , executive function ( e.g. , set - shifting ) , working memory , language , and memory / new verbal learning . \n results showed that pda+ performed significantly worse on the digit span forward and backward test and part b of the trail making tas

In [5]:
!python -m spacy download en_core_web_lg
!pip install scispacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_sm-0.5.1.tar.gz

2023-08-05 04:11:34.448437: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-05 04:11:35.760730: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 436.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 kB 18.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 6.9 MB/s eta 0:00:00
  Using cached pybin

In [34]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import re
nlp = spacy.load("en_core_sci_sm")

In [35]:
# function to return a list of matched entities between abstract and source article

def entityMatch(ent, source):
  ent_split = ent.split()
  result = []
  for l in range(len(ent_split), 1, -1):
    for start_i in range(len(ent_split) - l + 1):
      sub_ent = " ".join(ent_split[start_i:start_i+l])
      if re.search(re.escape(sub_ent), source, re.IGNORECASE):
        result.append(sub_ent)
    if result:
      break
  if result:
    return result
  else:
    for token in ent_split:
      if token.lower() not in STOP_WORDS:
        if re.search(re.escape(token), source, re.IGNORECASE):
          result.append(token)
    return result
  #print([])
  return []

In [36]:
# function to calculate precision-source metric

def calcPrecS(sourcetext, predicted_abstract):
    art = nlp(sourcetext)
    abs = nlp(predicted_abstract)

    matched_entities = []
    entities_per_abs = []
    for e in abs.ents:

      entities_per_abs.append(e.text)
      #print(entities_per_abs)

      match_result = entityMatch(e.text, predicted_abstract)
      if match_result:
        matched_entities.append(match_result[0])
        #print(matched_entities)

    # print(entities_per_abs)
    #print(len(matched_entities))
    #print(len(entities_per_abs))
    precS = len(matched_entities)/len(entities_per_abs)
    return precS

In [44]:
# function to calculate precision-target and recall-target and F1-target metrics

def calcPrecT(abstract, predicted_abstract):
    generated_summ = nlp(predicted_abstract)
    abs = nlp(abstract)

    matched_entities = []
    entities_per_predicAbs = []
    entities_per_abs = []

    for e in abs.ents:
      entities_per_abs.append(e.text)

    for e in generated_summ.ents:

      entities_per_predicAbs.append(e.text)

      match_result = entityMatch(e.text, abstract)
      if match_result:
        matched_entities.append(match_result[0])

    #print(entities_per_predicAbs)
    #print(matched_entities)
    #print(entities_per_abs)

    precT = len(matched_entities)/len(entities_per_predicAbs)
    recallT = len(matched_entities)/len(entities_per_abs)
    F1T= 2*precT*recallT/(precT+recallT)
    return [precT, recallT,F1T,entities_per_abs,entities_per_predicAbs,matched_entities]

In [33]:
print(calcPrecT(data_final['abstract'][0],data_final['predicted_abstract'][0]))

['quality of life', "parkinson's disease", 'cognitive status', 'motor deficits', 'apathy', 'depression', 'anxiety', 'depression', 'coexist', 'patients', 'research', 'anxiety', 'depression', 'prominent', 'prevalent', 'mood disorder', 'pd setting', 'anxiety', 'impact', 'cognition', 'neural basis', 'treatment', 'meager', 'lags', 'depression.methodswe', 'cross', 'sectional study', 'subjects', 'anxiety', 'thirty', 'subjects', 'anxiety', 'neurological evaluation', 'participants', 'yahr', 'lm', 'immediate', 'retention', 'hads', 'attentional set', 'shifting', 'digit span tests', 'assessed', 'working memory', 'attention', 'lesser', 'logical memory test', 'assessed', 'memory', 'verbal learning', 'pda+ group', 'levels', 'anxiety', 'mental state assessment', 'task force', 'iii', 'hads', 'rpd+ group', 'performance', 'digit span forward subtest', 'p2.033', 'backward subtest', 'wechsler memory scale- iii', 'tmt', 'b', 'p3.39).conclusionsthese', 'findings', 'anxiety', 'processing capacity', 'impair pr

In [28]:
!pip install tqdm

In [46]:
import csv
from tqdm import trange
from tqdm import tqdm
#finding BERT score
l=len(data_final)
with open('Test_EntityScore.csv', mode='w') as csv_file:
  fieldnames = ['article','precS','precT','recallT','F1-T','abstract_entity','predicted_entity','matched_entity']
  #fieldnames = ['Article', 'Actual Summary','Predective Summary']
  writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
  writer.writeheader()

  #variables to calculate sum of BERT score
  sum_precS=0
  sum_precT=0
  sum_recallT=0
  sum_F1_T=0
  for i in tqdm(range(l)):

    sample_row = data_final.iloc[i]
    article_Text=sample_row['article']
    model_output = sample_row['predicted_abstract']
    ref_output = sample_row['abstract']


    #calculating Precision of scource-score
    precS=calcPrecS(article_Text,model_output)
    targetscore=calcPrecT(ref_output,model_output)


    precS=round(precS,4)
    precT=round(targetscore[0],4)
    recallT=round(targetscore[1],4)
    F1T=round(targetscore[2],4)

    writer.writerow({'article':article_Text, 'precS':precS,'precT':precT,'recallT':recallT,'F1-T':F1T,'abstract_entity':targetscore[3],'predicted_entity':targetscore[4],'matched_entity':targetscore[5]})

    sum_precS=sum_precS+precS
    sum_precT=sum_precT+precT
    sum_recallT=sum_recallT+recallT
    sum_F1_T=sum_F1_T+F1T

 #finding average Bert score
  Avg_precS =sum_precS/l
  Avg_precT=sum_precT/l
  Avg_recallT =sum_recallT/l
  Avg_F1_T=sum_F1_T/l

  #print average  bert score
  print('\n')
  Avg_precS = round(Avg_precS,4)
  Avg_precT = round(Avg_precT,4)
  Avg_recallT = round(Avg_recallT,4)
  Avg_F1_T = round(Avg_F1_T,4)
  print("Avg_sourcePrecision",Avg_precS)
  print("Avg_targetPrecision",Avg_precT)
  print("Avg_targetRecall", Avg_recallT)
  print("Avg_targetF1", Avg_F1_T )
#next one
# write all results in a text file
with open("EntityScore.txt","w") as f:
  f.write("Avg_sourcePrecision:" + str(Avg_precS) + "\n")
  f.write("Avg_targetPrecision:" + str(Avg_precT) +'\n')
  f.write("Avg_targetRecall:" + str(Avg_recallT) + "\n")
  f.write("Avg_targetF1:" + str(Avg_F1_T) + "\n")


100%|██████████| 250/250 [03:26<00:00,  1.21it/s]



Avg_sourcePrecision 0.9948
Avg_targetPrecision 0.5446
Avg_targetRecall 0.4531
Avg_targetF1 0.4686
